# WSI - Ćwiczenie 2

*Autor: Maksymilian Nowak*

### Cel ćwiczenia

Celem ćwiczenia jest zaimplementowanie algorytmu ewolucyjnego:
- z selekcją turniejową ($k=2$)
- krzyżowaniem jednopunktowym
- mutacją gaussowską
- sukcesją generacyjną

Należy również wykorzystać ten algorytm do znalezienia minimum sumy funkcji $f_1+f_2$, gdzie: 
$$f_1(x_1,y_1)=(x_1^2+y_1-11)^2+(x_1+y_1^2-7)^2$$
$$f_2(x_2,y_2)=2*x_2^2+1.05*x_2^4+\frac{x_2^6}{6}+x_2y_2+y_2^2$$
dla dziedziny $D=[-5,5]\times[-5,5]$, oraz zbadać wpływ dystrybucji populacji początkowej, prawdopodobieństwa mutacji oraz prawdopodobieństwa krzyżowania na współrzędne znalezionych minimów funkcji $f_1$.

## Implementacja algorytmu ewolucyjnego

Do implementacji algorytmu przyjąłem następujące założenia:
- Funkcja celu ma następującą postać: $g(x_1, x_2, y_1, y_2)=f_1(x_1,y_1)+f_2(x_2,y_2)$
- Osobnik jest postaci $(x_1,x_2,y_1,y_2)$
- Kryterium stopu w algorytmie będzie maksymalna liczba pokoleń

Na początku zdefiniuję badane funkcje oraz funkcję celu, będącą ich sumą:

In [1]:
def f1(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

def f2(x, y):
    return 2*x**2 + 1.05*x**4 + (1/6)*x**6 + x*y + y**2

def g(x1, x2, y1, y2):
    return f1(x1, y1) + f2(x2, y2)

Pozostałymi hiperparametrami algorytmu będą:
- prawdopodobieństwo mutacji
- wariancja używana do obliczania siły mutacji 
- prawdopodobieństwo krzyżowania
- wartość kary za wyjście poza planszę (czyli dziedzinę $D$)

Przed zaimplementowaniem algorytmu wyznaczę funkcję oceny - wartością zwracaną przez tę funkcję będzie wynik funkcji celu (im mniejszy, tym lepsza ocena). Dodatkowo kara za wyjście poza dziedzinę funkcji przez osobnika to dodanie 1000 do wyniku działania funkcji oceny.

In [10]:
def rate(genes):
    penalty = 0
    domain_conditions = [-5 <= gene <= 5 for gene in genes]
    if not all(domain_conditions):
        penalty += 1000
    return g(gene[0], gene[1], gene[2], gene[3]) + penalty

Następnie zdefiniuję funkcje odpowiedzialne za reprodukcję, krzyżowanie i mutację, z których będzie się składał algorytm ewoulucyjny.

In [42]:
import random
import numpy as np

# Reprodukcja - selekcja turniejowa (turnieje dwuosobnikowe)
def reproduction(population, rating):
    results = []
    for i, individual in enumerate(population):
        opponents = population.copy()
        opponents.pop(i)
        opponent = random.choice(opponents)
        if rating(individual) < rating(opponent):
            results.append(individual)
        else:
            results.append(opponent)
    return results

# Krzyżowanie jednopunktowe - każda para rodziców produkuje dwójkę dzieci
def crossover(population, crossover_prob):
    results = []
    parents_list = population.copy()
    for i in range(0, len(population) - 1, 2):
        crossover_value = random.uniform(0, 1)
        parent1, parent2 = random.sample(parents_list, 2)
        parents_list.remove(parent1)
        parents_list.remove(parent2)
        if crossover_value < crossover_prob:
            crossing_point = random.randint(1, len(parent1))
            child1, child2 = [
                parent1[:crossing_point] + parent2[crossing_point:],
                parent2[:crossing_point] + parent1[crossing_point:]
            ]
            results.append(child1)
            results.append(child2)
        else:
            results.append(parent1)
            results.append(parent2)
    # Jeśli populacja jest nieparzysta, przenieś rodzica bez pary do wyników
    if len(population) % 2 == 1:
        results.append(parents_list[0])
    return results

# Mutacja gaussowska
def mutation(population, variance, mutation_prob):
    results = []
    for individual in population:
        mutation_value = random.uniform(0, 1)
        if mutation_value < mutation_prob:
            results.append((individual + variance * np.random.normal(0, 1, len(individual))).tolist())
        else:
            results.append(individual)
    return results
